# MicroStrategy Debt Risk Analysis

Comprehensive analysis of MSTR's convertible debt structure and liquidation scenarios.

In [ ]:
import sys
sys.path.append('..')

from src.parsers.debt_parser import parse_debt_data, calculate_debt_metrics
from src.models.liquidation_calculator import LiquidationCalculator
from src.models.maturity_analysis import MaturityAnalyzer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
%matplotlib inline

## 1. Load and Parse Debt Data

In [ ]:
debt_df = parse_debt_data('../data/raw/DEBT/data.html')
debt_metrics = calculate_debt_metrics(debt_df)

print("Debt Structure Overview:")
print(f"Total Debt: ${debt_metrics['total_notional']:,.0f}M")
print(f"Weighted Avg Coupon: {debt_metrics['weighted_avg_coupon']:.3f}%")
print(f"Number of Bonds: {debt_metrics['num_bonds']}")
print(f"\nMaturity Range: {debt_metrics['nearest_maturity'].strftime('%Y-%m-%d')} to {debt_metrics['furthest_maturity'].strftime('%Y-%m-%d')}")

debt_df

## 2. Liquidation Analysis

In [ ]:
# Initialize calculator with current data
btc_holdings = 447_470
btc_price = 100_000

calc = LiquidationCalculator(
    btc_holdings=btc_holdings,
    btc_price=btc_price,
    total_debt=debt_metrics['total_notional'],
    annual_interest=debt_metrics['total_notional'] * (debt_metrics['weighted_avg_coupon'] / 100)
)

print("Current Risk Metrics:")
print(f"LTV Ratio: {calc.calculate_ltv_ratio():.2%}")
print(f"Collateral Coverage: {calc.calculate_collateral_coverage():.2f}x")
print(f"\nLiquidation Prices:")
for ltv in [0.50, 0.65, 0.85]:
    liq_price = calc.calculate_liquidation_price(ltv)
    print(f"  At {ltv:.0%} LTV: ${liq_price:,.0f}")

## 3. Stress Testing

In [ ]:
stress_df = calc.stress_test_scenarios()
stress_df

## 4. Maturity Analysis

In [ ]:
analyzer = MaturityAnalyzer(debt_df)

print("Maturity Schedule:")
analyzer.get_maturity_schedule()

In [ ]:
print("Maturity Wall:")
analyzer.calculate_maturity_wall()

## 5. Sensitivity Analysis

How does LTV change across different BTC prices?

In [ ]:
# Generate LTV across price range
prices = np.arange(20000, 200000, 5000)
ltvs = [calc.calculate_ltv_ratio(p) * 100 for p in prices]

plt.figure(figsize=(12, 6))
plt.plot(prices, ltvs, linewidth=2, label='LTV Ratio')
plt.axhline(y=50, color='yellow', linestyle='--', label='Caution (50%)')
plt.axhline(y=85, color='red', linestyle='--', label='Danger (85%)')
plt.axvline(x=btc_price, color='green', linestyle='--', alpha=0.7, label=f'Current (${btc_price:,})')

plt.xlabel('Bitcoin Price ($)', fontsize=12)
plt.ylabel('LTV Ratio (%)', fontsize=12)
plt.title('LTV Sensitivity to Bitcoin Price', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Key Findings

### Summary:
1. **Current Position**: MSTR's LTV is very conservative at ~18%, providing substantial buffer
2. **Liquidation Risk**: BTC would need to drop 78% before reaching 85% LTV threshold
3. **Debt Maturity**: Bonds mature between 2028-2032, providing time for BTC appreciation
4. **Conversion Potential**: At current BTC trajectory, most bonds likely convert to equity (no cash refinancing needed)